#VoiceRecognition wav file exploration and understanding

This notebook is to understand the data format and layout of recording using Python Wave, wav files, and numpy arrays all representing the same underlying audio

##1) Record some audio to investigate with

###Prep
Import what we need

In [1]:
import wave
import numpy as np
import struct
from array import array
import sys
import pyaudio

###Config for the stream

In [2]:
THRESHOLD = 150  # Originally 500, but new is_silent() works better with this value
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 44100

RECORD_SECONDS = 5

###Set up and record a stream from the mic


In [4]:
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK_SIZE * RECORD_SECONDS)):
    stream_data = stream.read(CHUNK_SIZE)
    frames.append(stream_data) # 2 bytes(16 bits) per channel
    # print len(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

* recording
* done recording


###Display some information on what we've recorded

*len(data) will show us how much data is captured durectly from stream.read() based on our CHUNK_SIZE
*len(frames) will show us how much data has been recorded directly into our frames buffer

In [5]:
print "Data recorded for each stream.read(): ", len(stream_data)
print "Total number of frames captured in frames: ", len(frames)


Data recorded for each stream.read():  2048
Total number of frames captured in frames:  215


In [6]:
print "Number of CHUNKs recorded in total:", len(stream_data) * len(frames)

Number of CHUNKs recorded in total: 440320


Now display some of the data that we've captured to get a flavour of what it looks like

In [7]:
print "Data from stream.read():"
print stream_data
print "\nData in frames:"
print frames[:2]

Data from stream.read():
                            ����               ����      ����  ��                ��      ��  ����  ��    ��       ��                            ��          ������      ��  ��    ������  ��        ����         ������������    ������������      ����������������  ��     ����������������������������������  ��          ��    ��    ��  ��       ��        ����                                                                             ����                                                ��                    ��  ��          ��    ��              ������  ��      ��      ��������  ��            ��  ����      ��      ����  ��       ��    �� ��    ����    ��        ����                     ��                                                                     ��                    ����             ��               ��           ����             ��                    

###Write this file to disk for later

In [8]:
wf = wave.open("testing.wav", 'wb')
wf.setnchannels(1)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes('b'.join(frames))
wf.close()

###Using Numpy
Let's view some of this data in numpy as that's have we're going to manipulate it.

1) First we need to convert the frames data from integer into string (or can we get around this without as the stream.read() call returns a string in data???

In [9]:
fs = ",".join(str(f) for f in frames)
audio_data = np.array(struct.unpack_from(
        "%dh" % len(frames)*CHUNK_SIZE, 
        fs))

# Now print some stats about what we've just created in np and a snippet of the numpy array
print audio_data.shape
print audio_data[4000:5000]

(220160,)
[    0  -512   255     0   512     0  -256   255     0   256   512   256
     0   256   256   512     0   256   512   512   512   512     0   512
   512     0   256   256     0     0     0     0   512     0     0   256
   512   512   256     0   256     0     0  -256  -257   255   256     0
     0   512     0  -256   255     0     0     0  -512   255   256     0
  -256  -257    -1  -257  -257    -1  -513  -513  -513  -257   255  -256
  -513  -257  -257  -257    -1  -513  -513  -257    -1   255  -512    -1
   255     0     0   256     0  -256   255  -256  -257    -1   255  -768
   255 11264    -1    -2    -2    -2     0     0     0     0    -1     0
     0     0     0     0     0     0     2     0    -2    -1     0     1
     0    -1    -2    -3    -1    -2     0    -1    -2    -1    -1     0
     0    -1    -2    -2    -1    -2    -2    -1    -1     0     0     0
    -1     0     1     0     1     2     1     2     2     1     0     1
     0     0     0     0     0     2     

##2) Now start to process features
In this part of the code we are going to extract the features from the audio following a [Mel Frequency Cepstrum](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum) algorithm so that we can analyse them.


i) We are now going to use my modified version of [Hanoop Hallur's](https://github.com/anooprh/PyOhio-Prsesentation) code to extract features from the audio into a vector. For testing purposes we'll also write the feature vectors to a file in case we want to use them later for training or testing purposes.

In [10]:
from audioUtils import extractfeatures as features
from audioUtils import audioconfig as config
import time

sample_time = time.time()
features = features.extractfeatures(audio_data)
features_filename = "features.data/features-" + str(sample_time) + ".data"
print "Writing features to " + features_filename
np.savetxt(features_filename, features)

Writing features to features.data/features-1460958813.67.data


ii) Let's just get some basic stats about the data, graph something and see what it looks like!!
Perhaps use PCA to reduce the dimensionality of it.

Note that in extractfeatures.py we take the first 13 features, but we end up with 39. This is because we do deltas, and a delta of the delta.

In [11]:
print features.shape
print features

(398, 39)
[[-6.8234955  -0.96140744 -0.9989845  ..., -0.87004955 -0.57606942
   0.69646385]
 [-0.44990177  1.01759058 -0.9117311  ...,  2.00336778  1.53612203
   0.84517799]
 [ 0.42822127  2.10246006  0.04535481 ...,  0.03725601  0.54438953
  -0.79864392]
 ..., 
 [-0.30982433  0.19325759  0.90519868 ..., -0.08499086 -0.7365944
  -0.01158036]
 [-0.04073376  0.05950993  0.43918158 ..., -0.24703147 -0.56882291
  -0.06505578]
 [ 0.43481139 -0.94560027 -0.84772218 ...,  0.32178618  0.44750741
   0.51584874]]


##TODO - Load features from an existing file so we don't have to record again!!

In [ ]:
import matplotlib.pyplot as plt

#Start by transposing features so that the graph is longer in the x-direction
features_t = features.transpose()

#Now split out into original, delta and delta-delta


plt.figure(1)
plt.clf()
plt.imshow(features_t)
plt.title('Initial graph of a vector containing "Hello"')
plt.show()